# Langchain Demo with telegram data

In [13]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [17]:
# Index initialization
from semantic_search_generator import SemanticSearchGenerator

channel_id = "@runonflux"
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
INDEX_NAME = "telegram-embeddings"

generator = SemanticSearchGenerator(model_name)

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key=os.environ["PINECONE_APIKEY"],
    environment="us-west1-gcp"
)

# connect to index
index = pinecone.Index(INDEX_NAME)

In [33]:
from langchain.chains import LLMChain

prompt_template = """Use the chat messages (not sorted in any particular order) below to answer the given user query:
    messages_list: {messages}
    query: {query}
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["messages", "query"])
llm = OpenAI(temperature=0)
chain = LLMChain(llm=llm, prompt=PROMPT)


In [36]:
def search_results(query, limit=50):
    query_emb = generator.encode_messages(query)

    results = index.query(
      vector=query_emb.tolist(),
      top_k=limit,
      include_values=False,
      include_metadata=True
    )

    messages = []
    for item in results["matches"]:
        # print(f"\nscore {item['score']}")
        # print(item["metadata"]["clean_message"])
        messages.append(item["metadata"]["clean_message"])

    return messages
    

In [40]:
query = "what is good about this project?"
messages = search_results(query)
inputs = [{"message": msg} for _, msg in zip(range(len(messages)), messages)]
chain.run({"messages":inputs, "query":query})

'\nAnswer: This project has a lot of good points. It is a proof of useful work, providing and renting infrastructure, and has strategic partnerships with Octominer and Neurai. It is also launching the virtual shared space called the Metaverse and is pushing the boundaries of innovation by connecting the power of AI technology with the limitless potential of the Web3 Cloud.'

In [42]:
query = "what do users complain this project?"
messages = search_results(query)
inputs = [{"index": i, "message": msg} for i, msg in zip(range(len(messages)), messages)]
chain.run({"messages":inputs, "query":query})

"\nUsers have complained about people talking negatively about the project, people not doing enough for the project, people buying high and selling low, negative sentiment due to someone's opinions, and lack of collaboration opportunities."